In [27]:
import nltk
import os
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
# nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

In [28]:
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
          dict_freq[word] = words.count(word)
    return dict_freq

def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
        lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words

def stem_words(words):
    stemmed_words = []
    for word in words:
        stemmed_words.append(stemmer.stem(word))
    return stemmed_words

def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb

def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf

def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf

def tf_idf_score(tf,idf):
    return tf*idf

def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf

def sentence_importance(sentence,dict_freq,sentences):
    sentence_score = 0
    sentence = remove_special_characters(str(sentence))
    sentence = re.sub(r'\d+', '', sentence)
    pos_tagged_sentence = []
    no_of_sentences = len(sentences)
    pos_tagged_sentence = pos_tagging(sentence)
    for word in pos_tagged_sentence:
         if word.lower() not in Stopwords and word not in Stopwords    and len(word)>1:
            word = word.lower() 
            word = wordlemmatizer.lemmatize(word)
            sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
    return sentence_score


In [41]:
# file = 'Translated_text_mobicontrol_Page01_JP_to_En.txt'
# file = open(file , 'r')
# text = file.read()

text = 'Generating Reports SOTI MobiControl provides an extensive range of device and system reports to analyze your SOTI MobiControl deployment. The default reports cover many aspects of SOTI MobiControl, including but not limited to, device status and information, deployment server status and information, and administrative topics. Furthermore, you can create custom reports on other statistics that SOTI MobiControl tracks. The deployment server database is a standard SQL database that can be queried using a custom query. For more information on creating custom reports, contact SOTI MobiControl support. Reports can be generated in the following formats: Adobe PDF (.pdf) Comma separated values (.csv) Excel (.xls) Plain text (.txt) You can set SOTI MobiControl to automatically generate and disseminate reports. If you experience difficulties generating reports, make sure you have correctly activated SOTI MobiControl and that you have enabled TCP/IP in your SQL Server network configuration. Additionally, authentication of SQL Server must be performed using SQL Server Authentication; Windows Authentication is no longer supported. Tip: For a more flexible alternative, consider executing an Advanced Search and exporting the search results to a .csv file.'

tokenized_sentence = sent_tokenize(text)
#print(tokenized_sentence)

text = remove_special_characters(str(text))
text = re.sub(r'\d+', '', text)
tokenized_words_with_stopwords = word_tokenize(text)

tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
tokenized_words = [word for word in tokenized_words if len(word) > 1]
tokenized_words = [word.lower() for word in tokenized_words]
tokenized_words = lemmatize_words(tokenized_words)
word_freq = freq(tokenized_words)
input_user = int(input('Percentage of information to retain(in percent):'))
no_of_sentences = int((input_user * len(tokenized_sentence))/100)

print(no_of_sentences)
c = 1
sentence_with_importance = {}
for sent in tokenized_sentence:
    sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)    
    sentence_with_importance[c] = sentenceimp    
    c = c+1

sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
cnt = 0
summary = []
sentence_no = []

for word_prob in sentence_with_importance:
    if cnt < no_of_sentences:
        sentence_no.append(word_prob[0])
        cnt = cnt+1
    else:
        break
    
sentence_no.sort()
cnt = 1
for sentence in tokenized_sentence:
    if cnt in sentence_no:
        summary.append(sentence)
    cnt = cnt+1
    
summary = " ".join(summary)
print("\n")
print("Summary:")
print(summary)
outF = open('summary.txt',"w")
outF.write(summary)

Percentage of information to retain(in percent):50
4


Summary:
The default reports cover many aspects of SOTI MobiControl, including but not limited to, device status and information, deployment server status and information, and administrative topics. The deployment server database is a standard SQL database that can be queried using a custom query. For more information on creating custom reports, contact SOTI MobiControl support. Tip: For a more flexible alternative, consider executing an Advanced Search and exporting the search results to a .csv file.


497

In [25]:
len(tokenized_sentence)

1006

In [34]:
word_freq


{'you': 4,
 'set': 16,
 'title': 1,
 'customizer': 2,
 'subtitle': 1,
 'setting': 19,
 'order': 6,
 'mobicontrol': 23,
 'done': 1,
 'following': 12,
 'creating': 4,
 'required': 10,
 'connection': 2,
 'profile': 6,
 'case': 7,
 'terminal': 119,
 'o': 20,
 'target': 14,
 'premise': 2,
 'server': 28,
 'matter': 4,
 'io': 14,
 'device': 15,
 'macos': 2,
 'computer': 2,
 'add': 6,
 'active': 4,
 'directory': 4,
 'domain': 2,
 'service': 4,
 'adfs': 4,
 'federation': 2,
 'azure': 2,
 'ad': 2,
 'essential': 3,
 'want': 10,
 'manage': 4,
 'window': 10,
 'modern': 3,
 'the': 7,
 'one': 10,
 'company': 5,
 'payment': 2,
 'android': 12,
 'shared': 2,
 'plurality': 4,
 'employee': 2,
 'mandatory': 2,
 'apple': 8,
 'google': 8,
 'purpose': 4,
 'obtaining': 2,
 'microsoft': 2,
 'administrator': 3,
 'account': 9,
 'site': 2,
 'application': 24,
 'managed': 6,
 'play': 4,
 'purchase': 2,
 'app': 24,
 'store': 7,
 'authority': 3,
 'product': 4,
 'sign': 2,
 'catalog': 10,
 'if': 4,
 'distribute': 6,
 

In [35]:
sentence_with_importance

[(56, 0.18470475383461588),
 (25, 0.1800860943381154),
 (3, 0.16912135775974646),
 (57, 0.15611220452632302),
 (78, 0.13800608669492626),
 (37, 0.1339396505084311),
 (53, 0.1289201958360118),
 (42, 0.12849967614611124),
 (105, 0.12849967614611124),
 (50, 0.12548135986095904),
 (113, 0.12548135986095904),
 (1, 0.12430411901755785),
 (36, 0.12353215868899406),
 (24, 0.12230315027681463),
 (2, 0.11520869567480972),
 (54, 0.11346727236981374),
 (38, 0.10977965530137228),
 (70, 0.10732885061422692),
 (35, 0.10686481187440156),
 (26, 0.10613066998047081),
 (60, 0.10464126733722266),
 (68, 0.1039321478954247),
 (51, 0.10304926235607668),
 (58, 0.10297149828004457),
 (47, 0.10050762447783929),
 (110, 0.10050762447783929),
 (71, 0.09946958915171447),
 (32, 0.09826220807844367),
 (102, 0.09826220807844367),
 (61, 0.09729846055137703),
 (41, 0.09621232085876665),
 (104, 0.09621232085876665),
 (18, 0.09610817394283308),
 (94, 0.09610817394283308),
 (6, 0.09601936233113234),
 (82, 0.096019362331132